In [2]:
#%reload_ext autoreload
%load_ext autoreload

%autoreload 2


In [3]:
!pip show dill 

Name: dill
Version: 0.3.2
Summary: serialize all of python
Home-page: https://pypi.org/project/dill
Author: Mike McKerns
Author-email: None
License: 3-clause BSD
Location: /Users/Thugg/opt/miniconda3/envs/AIS/lib/python3.7/site-packages
Requires: 
Required-by: 


In [5]:
import dill

In [6]:
# dill.load_session('HyperParams.db')
dill.dump_session('HyperParams.db')

<h1>Imports <h1>

In [4]:
import sys
import os
print(sys.executable) 
print(sys.version) 
print(sys.version_info)
print(os.getcwd())
print(sys.path)
#sys.path.pop()
sys.path.append("/Users/Thugg/opt/miniconda3/envs/AIS/lib/python3.7/site-packages")

/Library/Frameworks/Python.framework/Versions/3.7/bin/python3
3.7.2 (v3.7.2:9a3ffc0492, Dec 24 2018, 02:44:43) 
[Clang 6.0 (clang-600.0.57)]
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
/Users/Thugg/Documents/UM/Summer_2020/Jupyter
['/Users/Thugg/Documents/UM/Summer_2020/Jupyter', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Users/Thugg/Library/Python/3.7/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions', '/Users/Thugg/.ipython']


In [4]:
import dill
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sn
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.base import clone
from zlib import crc32
from drop_collinear import drop_collinear
from run_rf import run_rf
from stratx import *
from stratx.featimp import *
from functools import reduce

In [13]:
#Read in CSV from Clean_CSV.pynb
csv_WCT = pd.read_csv("WCT_Clean.csv")
csv_RBT = pd.read_csv("RBT_Clean.csv")

In [21]:
X_wct = csv_WCT.drop('weightedPWCT', axis=1)
y_wct = csv_WCT['weightedPWCT']
X_rbt = csv_RBT.drop('weightedPRBT',axis=1)
y_rbt = csv_RBT['weightedPRBT']

In [22]:
#Drop Collinear Columns
X_wct_collinear = drop_collinear(X_wct,0.7)
X_rbt_collinear = drop_collinear(X_rbt,0.7)


<h1>Tune HyperParameters- WCT<h1>

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_wct_collinear,
                                                    y_wct,
                                                   test_size = 0.2,
                                                   random_state = 12)

In [70]:
imp_rf_wct = RandomForestRegressor()
imp_rf_wct.fit(X_train,y_train)

RandomForestRegressor()

In [161]:
#Tune hyper parameters before running feature importances

#First define lists of different parameter values to try
n_estimators = [int(x) for x in np.linspace(200,2000,10)]

max_features = [int(x) for x in np.linspace(2,8,7)]

max_depth = [int(x) for x in np.linspace(10,110,11)]

max_depth.append(None)

min_samples_split = [2,5,10]

min_samples_leaf = [1,2,4]

bootstrap = [True,False]


#Plug into a random grid for RandomizedSearchCV
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': [2, 3, 4, 5, 6, 7, 8],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [162]:
#Now use RandomizedSEarchCV
rf_random = RandomizedSearchCV(estimator = imp_rf_wct,
                               param_distributions = random_grid,
                               n_iter = 100,
                               cv = 50,
                               verbose = 2,
                               random_state = 12,
                               n_jobs = -1)
rf_random.fit(X_train,y_train)

Fitting 50 folds for each of 100 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed: 18.9min finished


RandomizedSearchCV(cv=50, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': [2, 3, 4, 5, 6, 7, 8],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=12, verbose=2)

In [125]:
#Base model MAE and CV score
predictions = imp_rf_wct.predict(X_test)
errors = abs(predictions - y_test)
cv = cross_val_score(imp_rf_wct,X_wct,y_wct,scoring = 'neg_mean_absolute_error',cv=5)

print("Mean Absolute Error: ", round(np.mean(errors),2),'%')
print("Mean CV Score: " ,round(np.mean(cv),2))

Mean Absolute Error:  12.87 %
Mean CV Score:  -11.14


In [163]:
rf_random.best_params_


{'n_estimators': 1200,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 5,
 'max_depth': 110,
 'bootstrap': False}

In [165]:
#Tuned MAE and CV score
best_search = rf_random.best_estimator_
predictions = best_search.predict(X_test)
errors = abs(predictions - y_test)
cv = cross_val_score(best_search,X_wct,y_wct,scoring = 'neg_mean_absolute_error',cv=50)

print("Mean Absolute Error: ", round(np.mean(errors),2),'%')
print("Mean CV Score: " ,round(np.mean(cv),2))

Mean Absolute Error:  11.28 %
Mean CV Score:  -10.95


In [147]:
#View best parameters
rf_random.best_params_



#***** BEST MODEL per random grid search*****
# {'n_estimators': 1200,
#  'min_samples_split': 10,
#  'min_samples_leaf': 1,
#  'max_features': 5,
#  'max_depth': 110,
#  'bootstrap': False}



#*Other results:*
# {'n_estimators': 1400,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 'sqrt',
#  'max_depth': 30,
#  'bootstrap': False}




# {'n_estimators': 600,
#  'min_samples_split': 10,
#  'min_samples_leaf': 1,
#  'max_features': 7,
#  'max_depth': 80,
#  'bootstrap': False}

{'n_estimators': 600,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 7,
 'max_depth': 80,
 'bootstrap': False}

In [168]:
#Do Grid Search around best parameters


param_grid = {'n_estimators': [600,700,800,900,1100,1200],
              'min_samples_split': [8,9,10,11,12],
              'min_samples_leaf': [1,2],
              'max_features': [3,4,5,6,7,8],
              'max_depth': [60,70,80,90,100,110],
              'bootstrap': [True,False]}
grid_rf = RandomForestRegressor()

grid_search = GridSearchCV(estimator=grid_rf,
                           param_grid = param_grid,
                           cv = 5, n_jobs = -1, verbose = 2)

In [169]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 4320 candidates, totalling 21600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-1)]: Done 6829 tasks      | elapsed: 26.3min
[Parallel(n_jobs=-1)]: Done 7922 tasks      | elapsed: 30.5min
[Parallel(n_jobs=-1)]: Done 9097 tasks      | e

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [60, 70, 80, 90, 100, 110],
                         'max_features': [3, 4, 5, 6, 7, 8],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [8, 9, 10, 11, 12],
                         'n_estimators': [600, 700, 800, 900, 1100, 1200]},
             verbose=2)

In [170]:
grid_search.best_params_


#*BEST MODEL TO BE USED HEREFORTH*
# {'bootstrap': False,
#  'max_depth': 70,
#  'max_features': 8,
#  'min_samples_leaf': 1,
#  'min_samples_split': 10,
#  'n_estimators': 600}

{'bootstrap': False,
 'max_depth': 70,
 'max_features': 8,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 600}

In [171]:
#Tuned MAE and CV score
best_grid = grid_search.best_estimator_
predictions = best_grid.predict(X_test)
errors = abs(predictions - y_test)
cv = cross_val_score(best_grid,X_wct,y_wct,scoring = 'neg_mean_absolute_error',cv=5)

print("Mean Absolute Error: ", round(np.mean(errors),2),'%')
print("Mean CV Score: " ,round(np.mean(cv),2))

Mean Absolute Error:  11.11 %
Mean CV Score:  -10.13


<h1>Now Tune for RBT <h1>
    

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_rbt_collinear,
                                                    y_rbt,
                                                   test_size = 0.2,
                                                   random_state = 12)

In [24]:
imp_rf_rbt = RandomForestRegressor()
imp_rf_rbt.fit(X_train,y_train)

RandomForestRegressor()

In [25]:
#Tune hyper parameters before running feature importances

#First define lists of different parameter values to try
n_estimators = [int(x) for x in np.linspace(200,2000,10)]

max_features = [int(x) for x in np.linspace(2,8,7)]

max_depth = [int(x) for x in np.linspace(10,110,11)]

max_depth.append(None)

min_samples_split = [2,5,10]

min_samples_leaf = [1,2,4]

bootstrap = [True,False]


#Plug into a random grid for RandomizedSearchCV
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': [2, 3, 4, 5, 6, 7, 8],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [26]:
#Now use RandomizedSEarchCV
rf_random = RandomizedSearchCV(estimator = imp_rf_wct,
                               param_distributions = random_grid,
                               n_iter = 100,
                               cv = 5,
                               verbose = 2,
                               random_state = 12,
                               n_jobs = -1)
rf_random.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.9min finished


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': [2, 3, 4, 5, 6, 7, 8],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=12, verbose=2)

In [28]:
rf_random.best_params_
# {'n_estimators': 1200,
#  'min_samples_split': 10,
#  'min_samples_leaf': 2,
#  'max_features': 6,
#  'max_depth': 110,
#  'bootstrap': False}

{'n_estimators': 1200,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 6,
 'max_depth': 110,
 'bootstrap': False}

In [29]:
#Do Grid Search around best parameters


param_grid = {'n_estimators': [1000,1100,1200,1300],
              'min_samples_split': [9,10,11],
              'min_samples_leaf': [1,2,3,4],
              'max_features': [4,5,6],
              'max_depth': [100,110,120,130,140,150],
              'bootstrap': [True,False]}
grid_rf = RandomForestRegressor()

grid_search = GridSearchCV(estimator=grid_rf,
                           param_grid = param_grid,
                           cv = 5, n_jobs = -1, verbose = 2)

In [30]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 16.9min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed: 20.4min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed: 24.0min
[Parallel(n_jobs=-1)]: Done 6829 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done 7922 tasks      | elapsed: 32.4min
[Parallel(n_jobs=-1)]: Done 8640 out of 8640 | 

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [100, 110, 120, 130, 140, 150],
                         'max_features': [4, 5, 6],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [9, 10, 11],
                         'n_estimators': [1000, 1100, 1200, 1300]},
             verbose=2)

In [40]:
grid_search.best_estimator_

# {'bootstrap': False,
#  'max_depth': 120,
#  'max_features': 6,
#  'min_samples_leaf': 1,
#  'min_samples_split': 10,
#  'n_estimators': 1000}





RandomForestRegressor(bootstrap=False, max_depth=120, max_features=6,
                      min_samples_split=10, n_estimators=1000)

In [1]:
#Base model MAE and CV score
predictions = imp_rf_wct.predict(X_test)
errors = abs(predictions - y_test)
cv = cross_val_score(imp_rf_wct,X_test,y_test,scoring = 'neg_mean_absolute_error',cv=5)

print("Mean Absolute Error: ", round(np.mean(errors),2),'%')
print("Mean CV Score: " ,round(np.mean(cv),2))

NameError: name 'imp_rf_rbt' is not defined

In [43]:
#Tuned MAE and CV score
best_search = rf_random.best_estimator_
predictions = best_search.predict(X_test)
errors = abs(predictions - y_test)
cv = cross_val_score(best_search,X_test,y_test,scoring = 'neg_mean_absolute_error',cv=5)

print("Mean Absolute Error: ", round(np.mean(errors),2),'%')
print("Mean CV Score: " ,round(np.mean(cv),2))

Mean Absolute Error:  7.96 %
Mean CV Score:  -5.53


In [9]:
#Tuned MAE and CV score
best_grid = grid_search.best_estimator_
predictions = best_grid.predict(X_test)
errors = abs(predictions - y_test)
cv = cross_val_score(best_grid,X_test,y_test,scoring = 'neg_mean_absolute_error',cv=5)

print("Mean Absolute Error: ", round(np.mean(errors),2),'%')
print("Mean CV Score: " ,round(np.mean(cv),2))

Mean Absolute Error:  8.07 %
Mean CV Score:  -6.41
